<a href="https://colab.research.google.com/github/cdebruyn/PackageName/blob/master/ZindiMobileMoney.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd

In [24]:
train = pd.read_csv('training.csv')
X = train.iloc[:, :-1]
y = train.loc[:, 'FraudResult']
test = pd.read_csv('test.csv')
df = pd.concat([X, test], sort=False)
df.head()

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,UGX,256,ProviderId_6,ProductId_10,airtime,ChannelId_3,1000.0,1000,2018-11-15T02:18:49Z,2
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-20.0,20,2018-11-15T02:19:08Z,2
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,UGX,256,ProviderId_6,ProductId_1,airtime,ChannelId_3,500.0,500,2018-11-15T02:44:21Z,2
3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,UGX,256,ProviderId_1,ProductId_21,utility_bill,ChannelId_3,20000.0,21800,2018-11-15T03:32:55Z,2
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-644.0,644,2018-11-15T03:34:21Z,2


In [25]:
df.shape

(140681, 15)

In [29]:
for column in df.columns:
  print(column, ': ', len(np.unique(df[column].values)))

ProviderId :  6
ProductId :  27
ProductCategory :  10
ChannelId :  5
Amount :  2099
Value :  1880
PricingStrategy :  4
TransactionDate :  120


In [0]:
df.loc[:, 'TransactionDate'] = df.apply(lambda x: x['TransactionStartTime'][0:10], axis=1)

In [0]:
df = df.drop(['TransactionId', 'BatchId', 'AccountId', 'SubscriptionId', 'CustomerId', 'CurrencyCode', 'CountryCode', 'TransactionStartTime'], axis=1)

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 140681 entries, 0 to 45018
Data columns (total 8 columns):
ProviderId         140681 non-null object
ProductId          140681 non-null object
ProductCategory    140681 non-null object
ChannelId          140681 non-null object
Amount             140681 non-null float64
Value              140681 non-null int64
PricingStrategy    140681 non-null int64
TransactionDate    140681 non-null object
dtypes: float64(1), int64(2), object(5)
memory usage: 9.7+ MB


In [0]:
df.loc[:, 'PricingStrategy'] = df['PricingStrategy'].astype('object')
df = pd.get_dummies(df, drop_first=True)

In [32]:
print('Original data set dimensions:')
print('Train: ', train.shape)
print('Test: ', test.shape)
X = df[:len(train)]
test = df[len(train):]
print('Original data set dimensions:')
print('Train: ', X.shape)
print('Test: ', test.shape)

Original data set dimensions:
Train:  (95662, 16)
Test:  (45019, 15)
Original data set dimensions:
Train:  (95662, 168)
Test:  (45019, 168)


In [0]:
from sklearn.neighbors.nearest_centroid import NearestCentroid

In [0]:
nc = NearestCentroid()
nc.fit(X, y)
test_ids = pd.read_csv('test.csv')['TransactionId']
predictions = pd.DataFrame([test_ids, nc.predict(test)]).T
predictions.columns = ['TransactionId', 'FraudResult']
predictions.to_csv('submission.csv')